### 0. 라이브러리

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import warnings
import pickle
import zipfile
from io import BytesIO

warnings.filterwarnings(action='ignore')
API_KEY = 'd7d1be298b9cac1558eab570011f2bb40e2a6825'

headers= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
          'Accept-Encoding': '*', 'Connection': 'keep-alive'}

options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--headless')

### 1. 데이터 추출 함수 세팅

In [2]:
def get_driver():
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
# 보고서명, 일자로 검색해서 보고서 접수번호 추출
def get_rcept_no(report_nm, bgn_de, end_de) :
    rcept_no_list = []
    url = 'https://opendart.fss.or.kr/api/list.xml'
    params = {'crtfc_key': API_KEY
              , 'bgn_de' : bgn_de
              , 'end_de' : end_de
              , 'pblntf_detail_ty': 'B001'
              , 'last_reprt_at':'Y'}
    try:
        response = requests.get(url, params=params, headers=headers, verify=False)
        soup = BeautifulSoup(response.content, features='html.parser')
        total_page = soup.find('total_page').get_text()
        
        for i in range(1, int(total_page) + 1):
            params = {'crtfc_key': API_KEY
                      , 'bgn_de': bgn_de
                      , 'end_de': end_de
                      , 'pblntf_detail_ty': 'B001'
                      , 'page_no': str(i)
                      , 'last_reprt_at':'Y'}
            response = requests.get(url, params=params, headers=headers, verify=False)
            soup = BeautifulSoup(response.content, features='html.parser')
            for c in soup.find_all('list'):
                if report_nm in c.report_nm.get_text():
                    rcept_no_list.append(c.rcept_no.get_text())
    except Exception as e:
        print(e)
        rcept_no_list = []
        
    print('보고서명:', report_nm,', 보고서수: ', len(rcept_no_list))
    
    return rcept_no_list

##### 2020년 이전 데이터 유실 많아서 selenium 필요

In [4]:
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

# def get_cps_docu(rcept_no):
#     print(rcept_no)
#     driver = webdriver.Chrome('./chromedriver', chrome_options=options)
#     url = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo='+rcept_no
#     driver.get(url)
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     company_nm = soup.select('body > div.wrapper > div > div.header > div.top > div.nameWrap > span:nth-child(2)')[0].get_text()
#     iframe_src = driver.find_element(by=By.XPATH, value='//*[@id="ifrm"]').get_attribute("src")
#     response = requests.get(iframe_src, headers=headers)
#     soup = BeautifulSoup(response.content, features='html.parser')
    
#     nstk_estk = soup.select('td:-soup-contains("기타주식 (주)")') # 기타주식(주) 위치 찾기
#     if not nstk_estk:
#         row = {}
#     else:
#         nstk_estk_cnt = nstk_estk[0].next_sibling.next_sibling.get_text()  # 신주의 종류와 수
    
#         if nstk_estk_cnt.strip() in ('-', '0') :
#             row = {}
#         else:
#             rcept_dt = rcept_no[:8]
#             tbody1 = nstk_estk[0].parent.parent
#             fv_ps = tbody1.select('tr:nth-child(3) > td:nth-child(2)')[0].get_text() # 1주당 액면가액
#             ic_mthn = soup.select('td:-soup-contains("증자방식")')[0].next_sibling.next_sibling.get_text() # 증자방식
#             try:
#                 cvt = soup.select('td:-soup-contains("전환에 관한 사항")')[0].parent
#                 cvt_cdt = cvt.select('td:nth-child(3)')[0].get_text() # 전환조건
#                 temp_list = []
#                 for i in range(5):
#                     cvt = cvt.next_sibling.next_sibling
#                     temp_list.append(cvt.select('td:nth-child(2)')[0].get_text())
#                 cvt_prd = temp_list[0] # 전환청구기간
#                 cvt_knd = temp_list[1] # 전환주식종류
#                 cvt_cnt = temp_list[2] # 전환주식수
#                 vtr_info = temp_list[3] # 의결권
#                 dvd_info = temp_list[4] # 이익배당
#             except:
#                 cvt_cdt = '-'
#                 cvt_prd = '-'
#                 cvt_knd = '-'
#                 cvt_cnt = '-'
#                 vtr_info = soup.select('td:-soup-contains("의결권에 관한 사항")')[0].next_sibling.next_sibling.get_text()
#                 dvd_info = soup.select('td:-soup-contains("이익배당에 관한 사항")')[0].next_sibling.next_sibling.get_text()
#             pst_std_val = soup.select('td:-soup-contains("기타주식 (원)")')[0].next_sibling.next_sibling.get_text() # 신주발행가액
#             dc_rate = soup.select('td:-soup-contains("기준주가에 대한 할인율 또는 할증율")')[0].next_sibling.next_sibling.get_text() # 할인율 또는 할증율

#             row = {'발행사':company_nm, '공시일':rcept_dt, '신주의 종류와 수':nstk_estk_cnt, '1주당 액면가액':fv_ps, '증자방식': ic_mthn,
#                   '전환조건':cvt_cdt, '전환청구기간':cvt_prd, '전환주식종류': cvt_knd, '전환주식수': cvt_cnt, '의결권':vtr_info, '이익배당': dvd_info,
#                   '신주발행가액': pst_std_val, '할인율 또는 할증율(%)': dc_rate}

#     return row

In [ ]:
driver = get_driver()

In [7]:
def get_cps_docu(rcept_no):
    print(rcept_no)
#     driver = get_driver()
    url = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo='+rcept_no
    driver.get(url)
    driver.implicitly_wait(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    company_nm = soup.select('body > div.wrapper > div > div.header > div.top > div.nameWrap > span:nth-child(2)')[0].get_text()
    iframe_src = driver.find_element(by=By.XPATH, value='//*[@id="ifrm"]').get_attribute("src")
    response = requests.get(iframe_src, headers=headers)
    soup = BeautifulSoup(response.content, features='html.parser')
    
    nstk_estk = soup.select('td:-soup-contains("기타주식 (주)")') # 기타주식(주) 위치 찾기
    if not nstk_estk:
        row = {}
    else:
        nstk_estk_cnt = nstk_estk[0].next_sibling.next_sibling.get_text()  # 신주의 종류와 수
    
        if nstk_estk_cnt.strip() in ('-', '0') :
            row = {}
        else:
            rcept_dt = rcept_no[:8]
            tbody1 = nstk_estk[0].parent.parent
            fv_ps = tbody1.select('tr:nth-child(3) > td:nth-child(2)')[0].get_text() # 1주당 액면가액
            ic_mthn = soup.select('td:-soup-contains("증자방식")')[0].next_sibling.next_sibling.get_text() # 증자방식
            try:
                cvt = soup.select('td:-soup-contains("전환에 관한 사항")')[0].parent
                cvt_cdt = cvt.select('td:nth-child(3)')[0].get_text() # 전환조건
                temp_list = []
                for i in range(5):
                    cvt = cvt.next_sibling.next_sibling
                    temp_list.append(cvt.select('td:nth-child(2)')[0].get_text())
                cvt_prd = temp_list[0] # 전환청구기간
                cvt_knd = temp_list[1] # 전환주식종류
                cvt_cnt = temp_list[2] # 전환주식수
            except:
                cvt_cdt = '-'
                cvt_prd = '-'
                cvt_knd = '-'
                cvt_cnt = '-'
            try:
                vtr_info = soup.select('td:-soup-contains("의결권에 관한 사항")')[0].next_sibling.next_sibling.get_text()
                dvd_info = soup.select('td:-soup-contains("이익배당에 관한 사항")')[0].next_sibling.next_sibling.get_text()
            except:
                vtr_info = '-'
                dvd_info = '-'
            pst_std_val = soup.select('td:-soup-contains("기타주식 (원)")')[0].next_sibling.next_sibling.get_text() # 신주발행가액
            dc_rate = soup.select('td:-soup-contains("기준주가에 대한 할인율 또는 할증율")')[0].next_sibling.next_sibling.get_text() # 할인율 또는 할증율

            row = {'발행사':company_nm, '공시일':rcept_dt, '신주의 종류와 수':nstk_estk_cnt, '1주당 액면가액':fv_ps, '증자방식': ic_mthn,
                  '전환조건':cvt_cdt, '전환청구기간':cvt_prd, '전환주식종류': cvt_knd, '전환주식수': cvt_cnt, 
                  '의결권':vtr_info, '이익배당': dvd_info, '신주발행가액': pst_std_val, '할인율 또는 할증율(%)': dc_rate}

    return row

### 2. 보고서명, 일자로 검색 후 추출 - 보고서 원문 이용

In [5]:
# 최대 조회 가능 기간 3개월
bgn_de = '20150101'
end_de = '20150331'

In [6]:
# 보고서 접수번호 추출
rcept_name = '주요사항보고서(유상증자결정)'
rcept_no_list = []
rcept_no_list.extend(get_rcept_no(rcept_name, bgn_de, end_de))

보고서명: 주요사항보고서(유상증자결정) , 보고서수:  105


In [8]:
# 보고서 접수번호별 세부정보 추출
rows = []
for rcept in rcept_no_list:
    row = get_cps_docu(rcept)
    rows.append(row)

df = pd.DataFrame(rows)
df = df.dropna(subset=['발행사'])
df.head()

20150330001574


IndexError: list index out of range

In [19]:
# df.shape